In [22]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os
import nrrd as reader
#from sklearn.preprocessing import Normalizer
import cv2  
from skimage import morphology
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters

j = 5

In [23]:
def join(one, two):
    return os.path.join(one, two)

In [24]:
def check_path(path):
    count=0
    for files in sorted(os.listdir(path)):
        image_path = os.path.join(path,files)
        count+=1
    return count

In [25]:
# (patient count* 15) check_path function will return total patient count 
folder_path = join(os.getcwd(), "dataset")
test_path = join(os.getcwd(),"Test")
length = check_path(folder_path)
inputs = np.zeros((length*15,256,256, 1))
inputsFLE = np.zeros((length*15,256,256, 1))
outputs = np.zeros((length*15,256,256 , 9))

In [26]:
#Inhomogeneity Correction
def correct_roi(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    return image_c

In [27]:
def dcm_to_np(path):
    global dic_count
    try:
        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
        image = sitk.ReadImage(slice_filenames)
        for x in range(15):
            current_slice = image[:, :, x]
            current_array = sitk.GetArrayFromImage(current_slice)
            split_array = current_array[:,:-256]
            inputs[dic_count] = split_array[:, :, np.newaxis]
            dic_count+=1
        print(dic_count)
    except Exception as e:
        print(e)

In [28]:
def nrrd_to_np(meta_object):
    global ann_count
    try:
        data = sitk.GetArrayFromImage(meta_object)
        for p in range(15):
            outputs[ann_count] = np.expand_dims(np.where(data[p, :, :256] == 2, 1, 0), axis=2)
            ann_count+=1
        print(ann_count)
    except Exception as e:
        print(e)

In [29]:
dic_count = 0
ann_count = 0
try:
    for patient in sorted(os.listdir(folder_path)):
            patient_path = join(folder_path, patient)
            fle_folder = join(patient_path, "FLE")
            for fle in os.listdir(fle_folder):
                if(".nrrd" in fle):
                    flesegmentation = sitk.ReadImage(join(fle_folder, fle))
                    nrrd_to_np(flesegmentation)
                    
except Exception as e:
    print(e)

[Errno 2] No such file or directory: '/Users/jasontsang/Desktop/[TBD] RAMER/JuypterNotebook/dataset/123456/FLE'
